In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import parse_UK_Data, generate_large_distance_matrix, load_saved_here_data, save_here_data, get_results, generate_final_results


dataset = "../datasets/UK10_1.txt"
meta_data, distance_data, station_data = parse_UK_Data(dataset)
station_coordinates = pd.read_csv('../datasets/UK10_1_coordinates.csv', encoding='utf-8')
points_coordinate = station_coordinates[["lat", "long"]].to_numpy()

api = "Tfmt6Sxyn-nfinSHUyUo3xmVDWdtwAUyh6C8Y3HTZhU"

route1 = [[0,1,2,3], [4,5,6,7,8,9]]
route2 = [[0,2,3,5], [4,6,7,8,9,1]]
route3 = [[0,3,5,6], [4,7,8,9,1,2]]

In [2]:
meta_data["F-C Empty (l/100km)"] = 12.5
meta_data["F-C Full (l/100km)"] = 15
meta_data["Vehicle Weight"] = 3000
#distance_matrix_here,time_matrix_here,_ = generate_large_distance_matrix(points_coordinate,api)
#save_here_data("../datasets/", "UK10_1", distance_matrix_here, time_matrix_here)
distance_matrix_here,time_matrix_here = load_saved_here_data("../datasets/",dataset)

In [4]:
# def generate_final_results(results:list, comp_times:list, epochs:float):
    
#     results_summed = [pd.DataFrame(df.sum(axis=0)) for df in results]
    
#     data_df = pd.concat(results_summed, axis=1, ignore_index=True).T
#     data_df.drop(columns=["Travel Time hh:mm:ss", "Total Travel Time (s)"], inplace=True)
#     data_df["Computaion Time"] = comp_times
#     for axs in ["Avg Estimated Fuel Conspumtion (L/100km) (Hao et al.)", "Avg Estimated Fuel Conspumtion (L/100km) (Rakha et al.)", "Avg Speed (km/h)"]:
#         data_df[axs] = data_df[axs]/2
    
#     result_df = pd.concat({"Max":data_df.max(axis=0),
#                            "Min":data_df.min(axis=0),
#                            "Mean":data_df.mean(axis=0),
#                            "Std":data_df.std(axis=0)}, axis=1)
    
    
#     return data_df, result_df

data1 = get_results(route1, distance_matrix=distance_matrix_here, demand_data=station_data, meta_data=meta_data, travel_time_matrix=time_matrix_here)
data2 = get_results(route2, distance_matrix=distance_matrix_here, demand_data=station_data, meta_data=meta_data, travel_time_matrix=time_matrix_here)
data3 = get_results(route3, distance_matrix=distance_matrix_here, demand_data=station_data, meta_data=meta_data, travel_time_matrix=time_matrix_here)
comp_times = [100,200,300]
epochs = [10,20,30]
results_df = generate_final_results([data1, data2, data3], comp_times)


In [6]:
results_df

,Max,Min,Mean,Std
Total distance (km),1434.965000,959.571000,1238.857333,248.372466
Total load (kg),4147,3708,3904.000000,223.242021
Total Estimated Fuel Consumption (L) (Hao et al.),188.287565,128.019093,163.537190,31.544097
Avg Estimated Fuel Conspumtion (L/100km) (Hao et al.),1.317671,1.303608,1.312583,0.007796
Total Estimated Fuel Consumption (L) (Rakha et al.),137.532202,90.019132,117.944670,24.829714
Avg Estimated Fuel Conspumtion (L/100km) (Rakha et al.),0.965914,0.933774,0.953464,0.017250
Avg Speed (km/h),72.176725,64.848215,69.160040,3.832076
Computaion Time,300,100,200.000000,100.000000
